In [1]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

In [2]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm = ChatOpenAI(name="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)


In [ ]:
"""
3 x 4 + 12
[AIMessage id-1, ToolMessage id-1 value-1,
function id-2, result id-2 args=value]

"""

In [111]:
from dataclasses import dataclass

In [127]:
import json

In [138]:
@dataclass
class Function:
    name: str
    # id: str
    args: dict
    result: int = None

    def format(self):
        data = {
            "name": self.name,
            "args": self.args,
        }
        if self.result:
            data["result"] = self.result
        return json.dumps(data)

In [139]:
func1 = Function(name="multiply", args={"a": 110, "b": 2}, result="<value>")

In [141]:
func1.format()

'{"name": "multiply", "args": {"a": 110, "b": 2}, "result": "<value>"}'

In [145]:
func2 = Function(name="add", args={"a": "<value>", "b": 2})

In [147]:
func2.format()

'{"name": "add", "args": {"a": "<value>", "b": 2}}'

In [ ]:
"(2 + 1) x 2" 

In [50]:
examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)